In [1]:
import pandas as pd
import numpy as np

tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'NFLX']
folder = "/Users/pavelmanyakin/Desktop/Data/" #This should be changed to your folder with data
window_size = 60
Npi = 100
bandwidth = 0.1
all_prices = []
for ticker in tickers:
    file_path = f"{folder}{ticker}.csv"
    df = pd.read_csv(file_path, parse_dates=['Date'])
    price_col = 'Adjusted Close' if 'Adjusted Close' in df.columns else 'Adj Close'
    df = df[['Date', price_col]].rename(columns={price_col: ticker})
    all_prices.append(df)

data = all_prices[0]
for df in all_prices[1:]:
    data = data.merge(df, on='Date', how='inner')
data = data.set_index('Date')
data = data.loc["2010-01-01":"2019-12-31"]
log_returns = np.log(data / data.shift(1)).dropna()

def create_sequences(log_returns, window):
    sequences = []
    for i in range(len(log_returns) - window):
        seq = log_returns.iloc[i:i+window].values
        sequences.append(seq)
    return np.array(sequences)

data_series = create_sequences(log_returns, window_size)

def quartic_kernel(x, h):
    norm = np.linalg.norm(x) / h
    return ((1 - norm**2)**2) / h if norm < 1 else 0.0

def Fi(t, xti, x, xtip1, ti, tip1):
    dt1 = tip1 - t
    dt2 = tip1 - ti
    term1 = -np.linalg.norm(xtip1 - x)**2 / (2 * dt1)
    term2 = np.linalg.norm(xtip1 - xti)**2 / (2 * dt2)
    return np.exp(term1 + term2)

def estimate_drift(t, x, xi, data_series, ti, tip1, h):
    num = np.zeros_like(x)
    denom = 0.0
    for sample in data_series:
        if tip1 >= sample.shape[0]:
            continue
        xti = sample[ti]
        xtip1 = sample[tip1]
        Fi_val = Fi(t, xti, x, xtip1, ti, tip1)
        weight = Fi_val
        for j in range(ti):
            weight *= quartic_kernel(xi[j] - sample[j], h)
        num += weight * (xtip1 - x)
        denom += weight
    return (1 / (tip1 - t)) * num / (denom + 1e-8)

def sbts_simulation(data_series, Npi=100, h=0.1):
    M, N, d = data_series.shape
    x = [np.zeros(d)]
    for i in range(N - 1):
        y = x[i].copy()
        for k in range(Npi - 1):
            tpi_k = i + k / Npi
            drift = estimate_drift(tpi_k, y, np.array(x), data_series, ti=i, tip1=i+1, h=h)
            eps = np.random.randn(d)
            y = y + (1 / Npi) * drift + (1 / np.sqrt(Npi)) * eps
    return np.array(x)

simulated = sbts_simulation(data_series, Npi=Npi, h=bandwidth)
df_sim = pd.DataFrame(simulated, columns=tickers)
print(df_sim)

/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_81579/324559031.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(file_path, parse_dates=['Date'])
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_81579/324559031.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(file_path, parse_dates=['Date'])
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_81579/324559031.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(file_path, parse_dates=['Date'])
/var/folders/ml/w52lgnpd681d

        AAPL       MSFT      GOOG       AMZN      NFLX
0   0.000000   0.000000  0.000000   0.000000  0.000000
1   0.102575  -0.025845  0.160284  -0.106812 -0.154798
2  -0.017167  -0.005569  0.083400  -0.064650 -0.168095
3   0.695622   1.319096  1.891062   0.283761  0.000261
4  -0.989204   2.219756  1.115013   0.061883  0.051908
5   0.469553   2.948817  1.054555  -0.698881 -0.015663
6  -0.027145   3.384281  1.661316   0.284410 -0.687012
7   1.463075   5.819876  1.750459   1.470066 -2.121584
8   1.359555   5.300161  1.657420  -0.136435 -3.212955
9   2.074831   7.810663  1.590908   1.657258 -2.515924
10  1.776068   7.331574  1.256396   0.127545 -2.825140
11  1.995167   7.509798  2.665310   1.717409 -3.233903
12  0.677310   6.867972  1.096195   1.130959 -1.846538
13  1.863055   7.992514  1.751435  -0.418341 -0.761257
14  3.374654   6.739485 -0.265416   0.343211  0.192720
15  4.414668   6.399336 -1.762037   0.298711  2.239057
16  4.071955   6.761753 -1.643409  -0.033149  0.658237
17  3.3062

In [4]:
returns = df_sim.diff().dropna()

mu = returns.mean().values
cov = returns.cov().values
inv_cov = np.linalg.inv(cov)
ones = np.ones(len(mu))

w_sharpe = inv_cov @ mu
w_sharpe /= np.sum(w_sharpe)

portfolios = pd.DataFrame([w_sharpe],
                          columns=df_sim.columns,
                          index=['MaxSharpe'])
print(portfolios)

              AAPL      MSFT      GOOG      AMZN      NFLX
MaxSharpe  0.33375  0.335573  0.360659 -0.275404  0.245423


In [5]:
import os

tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'NFLX']
folder = "/Users/pavelmanyakin/Desktop/Data/" #This should be changed to your folder with data
start_date = '2020-01-01'
end_date = '2020-03-01'
initial_investment = 100000
weights = np.array([0.33375, 0.335573, 0.360659, -0.275404, 0.245423])

close_prices = pd.DataFrame()

for ticker in tickers:
    filepath = os.path.join(folder, f"{ticker}.csv")
    df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
    close_prices[ticker] = df['Adjusted Close']

close_prices = close_prices.loc[start_date:end_date]

start_prices = close_prices.iloc[0]
end_prices = close_prices.iloc[-1]

shares_bought = (initial_investment * weights) / start_prices

final_value = np.sum(shares_bought * end_prices)

print(f"Initial investment: ${initial_investment:,.2f}")
print(f"Final portfolio value after 2 months: ${final_value:,.2f}")
print(f"Return: {(final_value / initial_investment - 1) * 100:.2f}%")


Initial investment: $100,000.00
Final portfolio value after 2 months: $139,840.53
Return: 39.84%


/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_81579/511270854.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_81579/511270854.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_81579/511270854.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(filepath, parse_dates=['Dat